In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Combining Semantic & Keyword Search: A Hybrid Search Tutorial with Vertex AI Vector Search

Vector Search supports hybrid search, a popular architecture pattern in information retrieval (IR) that combines both semantic search and keyword search (also called token-based search). With hybrid search, developers can take advantage of the best of the two approaches, effectively providing higher search quality.

This tutorial explains the concepts of hybrid search, semantic search, and token-based search, and includes examples of how to set up token-based search and hybrid search.

This tutorial takes about 1 hour, mostly on waiting for Vector Search deployment (30 minutes).

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fembeddings%2Fhybrid-search.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/embeddings/hybrid-search.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/hybrid-search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

# Why does hybrid search matter?

As described in [Overview of Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview), semantic search with Vector Search can find items with semantic similarity by using queries.

Embedding models such as [Vertex AI embeddings](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings) build a vector space as a map of content meanings. Each text or multimodal embedding is a location in the map that represents the meaning of some content. As a simplified example, when an embedding model takes a text that discusses movies for 10%, music for 2%, and actors for 30%, it could represent this text with an embedding `[0.1, 0.02, 0.3]`. With Vector Search, you can quickly find other embeddings in its neighborhood. This searching by content meaning is called semantic search.

![Semantic search image](https://cloud.google.com/vertex-ai/docs/vector-search/images/semantic_search.gif)

Semantic search with embeddings and vector search can help make IT systems as smart as experienced librarians or shop staff. Embeddings can be used for tying different business data with their meanings; for example, queries and search results; texts and images; user activities and recommended products; English texts and Japanese texts; or sensor data and alerting conditions. With this capability, there's a wide variety of use cases for embeddings.

## Why combine semantic search with keyword-based search?

Semantic search doesn't cover all the possible requirements for information retrieval applications, such as [Retrieval-Augmented Generation (RAG)](https://cloud.google.com/use-cases/retrieval-augmented-generation). Semantic search can only find data that the embedding model can make sense of. For example, queries or datasets with arbitrary product numbers or SKUs, brand new product names that were added recently, and corporate proprietary codenames don't work with semantic search because they aren't included in the training dataset of the embedding model. This is called "out of domain" data.

In such cases, you would need to combine semantic search with keyword-based (also called token-based) search to form a hybrid search. With hybrid search, you can take advantage of both semantic and token-based search to achieve higher search quality.

One of the most popular hybrid search systems is Google Search. The service incorporated [semantic search in 2015 with RankBrain model](https://blog.google/products/search/how-ai-powers-great-search-results/), in addition to its token-based keyword search algorithm. With the introduction of hybrid search, Google Search was able to improve the search quality significantly by addressing the two requirements: search by meaning and search by keyword.

In the past, building a hybrid search engine was a complex task. Just like with Google Search, you have to build and operate two different kinds of search engines (semantic search and token-based search) and merge and rank the results from them. With hybrid search support in Vector Search, you can build your own hybrid search system with a single Vector Search index, customized to your business requirements.

## How token-based search works

How does token-based search in Vector Search work? After [splitting the text into tokens](https://huggingface.co/docs/transformers/en/tokenizer_summary) (such as words or sub-words), you can use popular sparse embedding algorithms such as [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), [BM25](https://en.wikipedia.org/wiki/Okapi_BM25), or [SPLADE](https://en.wikipedia.org/wiki/Learned_sparse_retrieval) to generate sparse embedding for the text.

A simplified explanation of sparse embeddings is that they are vectors that represent how many times each word or sub-word appears in the text. Typical sparse embeddings don't take semantics of the text into account.

![Sparse embeddings image](https://cloud.google.com/vertex-ai/docs/vector-search/images/sparse_embeddings.png)

There could be thousands of different words used in texts. Thus, this embedding usually has tens of thousands of dimensions, with only a few dimensions in them having non-zero values. This is why they're called "sparse" embeddings. The majority of their values are zeroes. This sparse embedding space works as a map of keywords, similar to an index of books.

In this sparse embedding space, you can find similar embeddings by looking at the neighborhood of a query embedding. These embeddings are similar in terms of the distribution of the keywords used in their texts.

![Token search image](https://cloud.google.com/vertex-ai/docs/vector-search/images/token_search.gif)

This is the basic mechanism of the token-based search with sparse embeddings. With hybrid search in Vector Search, you can mix both dense and sparse embeddings into a single vector index and run queries with dense embeddings, sparse embeddings, or both. The result is a combination of semantic search and token-based search results.

Hybrid search also provides shorter query latency compared to a token-based search engine with an [inverted index](https://en.wikipedia.org/wiki/Inverted_index) design. Just like vector search for semantic search, each query with dense or sparse embeddings finishes within milliseconds, even with millions or billions of items.


# Section 1. Create sparse embeddings

In this first section of the notebook you willgenerate sparse embeddings and build a Vector Search endpoint.


## Task 2. Install packages, configure the notebook.

The notebook uses Vertex AI SDK and Cloud Storage SDK, so you will need to install these. Once installed, you will then restart the kernel. After the kernel is restarted you will configure the notebook to work with your project and region assigned by Qwiklabs.


In [2]:
%pip install --upgrade --quiet --user google-cloud-aiplatform google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Install required packages
!pip install --upgrade --quiet google-cloud-aiplatform scikit-learn pandas

# Restart kernel after installs so your environment can access the new packages
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)



{'status': 'ok', 'restart': True}

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
# get project ID and set your region.
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
# Replace "[your-region]" with a valid Google Cloud region, e.g., "us-central1"
LOCATION = "us-west4" 
print(f"Project ID: {PROJECT_ID}")
print(f"Region: {LOCATION}")
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "qwiklabs-gcp-02-a8d4e63ebbf2"  # @param {type:"string"}
# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")


Project ID: qwiklabs-gcp-02-a8d4e63ebbf2
Region: us-west4


In [ ]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

## Task 3. Prepare a sample dataset

In this task you will prepare a data file to build an index for sparse embeddings, based on the data format described in [Input data format and structure](https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure).

In JSON, the data file looks like this:

```
{"id": "3", "sparse_embedding": {"values": [0.1, 0.2], "dimensions": [1, 4]}}
{"id": "4", "sparse_embedding": {"values": [-0.4, 0.2, -1.3], "dimensions": [10, 20, 20]}}
```

Each item should have a `sparse_embedding` property that has `values` and `dimensions` properties. Sparse embeddings have thousands of dimensions with a few non-zero values. This data format works efficiently because it contains the non-zero values only with their positions in the space.

As a sample dataset, we'll use the [Google Merch Shop](https://shop.merch.google/) dataset, which has about 200 rows of Google-branded goods.

The variable `corpus` holds a list of the 200 item names, such as "Google Sticker" or "Chrome Dino Pin". Then, the code passes them to the vectorizer by calling the `fit_transform()` function. With that, the vectorizer gets ready to generate sparse embeddings.

TF-IDF vectorizer tries to give higher weight to signature words in the dataset (such as "Shirts" or "Dino") compared to trivial words (such as "The", "a", or "of"), and counts how many times those signature words are used in the specified document. Each value of a sparse embedding represents a frequency of each word based on the counts. For more information about TF-IDF, see the later section "How do TF-IDF and TfidfVectorizer work?".

In this example, we use the basic word-level tokenization and TF-IDF vectorization for simplicity. In production development, you can choose any other options for tokenizations and vectorizations for generating sparse embeddings based on your requirements. For tokenizers, in many cases [subword tokenizers](https://www.tensorflow.org/text/guide/subwords_tokenizer) perform well compared to the word-level tokenization and are popular choices. For vectorizers, [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is popular as an improved version of TF-IDF. [SPLADE](https://en.wikipedia.org/wiki/Learned_sparse_retrieval) is another popular vectorization algorithm that takes some semantics for the sparse embedding.

### Get a sparse embedding
To make the vectorizer easier to use with Vector Search, we'll define a wrapper function, get_sparse_embedding():

In [ ]:
def get_sparse_embedding(text):
    # Transform Text into TF-IDF Sparse Vector
    tfidf_vector = vectorizer.transform([text])

    # Create Sparse Embedding for the New Text
    values = []
    dims = []
    for i, tfidf_value in enumerate(tfidf_vector.data):
        values.append(float(tfidf_value))
        dims.append(int(tfidf_vector.indices[i]))
    return {"values": values, "dimensions": dims}

This function passes the parameter "text" to the vectorizer to generate a sparse embedding. Then convert it to the `{"values": ...., "dimensions": ...}` format mentioned earlier for building a Vector Search sparse index.

You can test this function:

In [ ]:
text_text = "Chrome Dino Pin"
get_sparse_embedding(text_text)

### Create an input data file
For this example, we'll generate sparse embeddings for all 200 items.

In [ ]:
# set BUCKET_URI
BUCKET_URI = f"gs://{PROJECT_ID}"

In [ ]:
import numpy as np

# Define the 'matrix' variable before using it.
# In a real analysis, this would likely be loaded from a file or be the
# output of a previous data processing step.
matrix = np.array([[1, 2, 3.5, 4.0], 
                   [2, 5, 6.1, 7.5], 
                   [7, 8, 9.9, 9.1]])

# Now that 'matrix' is defined, you can safely call np.corrcoef().
# By default, np.corrcoef treats each row as a variable (e.g., a gene)
# and each column as an observation.
res = np.corrcoef(matrix)

# The resulting 'res' is a symmetric matrix where:
# - The diagonal elements are all 1 (correlation of a variable with itself).
# - The off-diagonal element res[i, j] is the correlation between row i and row j.
print("Correlation matrix:")
print(res)


Then, save them as a JSONL file `items.json` and upload to a Cloud Storage bucket.

In [ ]:
# output as a JSONL file and save to the GCS bucket
with open("items.json", "w") as f:
    for item in items:
        f.write(f"{item}\n")
! gsutil cp items.json $BUCKET_URI

# Section 2. Use hybrid search

With this section of the notebook, you will combine token-based search with semantic search to create hybrid search in Vector Search.

## Task 5. Create a index endpoint and sparse embedding index in Vector Search
Next, we'll build and deploy a sparse embedding index in Vector Search. This is the same procedure that is documented in the [Vector Search quickstart](https://cloud.google.com/vertex-ai/docs/vector-search/quickstart#create_an_index).

In [ ]:
# init the aiplatform package
from google.cloud import aiplatform

# --- Define your project and location ---
# Replace 'your-gcp-project-id' with your actual Google Cloud project ID.
# Replace 'us-central1' with the region you are working in.
PROJECT_ID = "qwiklabs-gcp-02-a8d4e63ebbf2"
LOCATION = "us-west4"

# Now, initialize the Vertex AI SDK with the defined variables
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print(f"Vertex AI SDK initialized successfully for project '{PROJECT_ID}' in location '{LOCATION}'.")


To use the index, you need to create an index endpoint. It works as a server instance accepting query requests for your index.

In [ ]:
from google.cloud import aiplatform
import uuid

# Generate a unique ID for this session
UID = uuid.uuid4().hex[:8]

# create `IndexEndpoint`
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-hybridsearch-index-endpoint-{UID}", public_endpoint_enabled=True
)

print(f"Index endpoint created: {my_index_endpoint.resource_name}")


## Task 6. Create the hybrid index and deploy it to the Endpoint

To build a hybrid index, each item should have both `embedding` (for dense embedding) and `sparse_embedding`:

In [ ]:
# get text embedding model
from vertexai.preview.language_models import TextEmbeddingModel
​
model = TextEmbeddingModel.from_pretrained("text-embedding-005")
​
​
# wrapper
def get_dense_embedding(text):
    return model.get_embeddings([text])[0].values
​
​
# test it
get_dense_embedding("Chrome Dino Pin")

# Load the data into the DataFrame again, as it may have been lost due to a kernel restart.
import json

# This code assumes the 'items' list has been generated by the preceding cells,
# and the 'BUCKET_URI' variable is defined in your notebook environment.

# 1. Define the local filename for the JSONL output
output_filename_local = "items_hybrid.jsonl"

# 2. Write the 'items' list to the local JSONL file
# The JSONL format requires one complete JSON object per line.
print(f"Writing {len(items)} items to local file: {output_filename_local}...")
with open(output_filename_local, "w") as f:
    for item in items:
        f.write(json.dumps(item) + "\n")
print("Local file created successfully.")

# 3. Upload the local JSONL file to your Google Cloud Storage bucket
# The !gsutil command is a simple and effective way to interact with GCS from a notebook.
print(f"Uploading {output_filename_local} to {BUCKET_URI}...")
!gsutil cp {output_filename_local} {BUCKET_URI}
print("Upload complete. You can now use this file in GCS to create your Vector Search index.")


# --- Define Helper Function for Sparse Conversion ---
def convert_sparse_to_dict(sparse_vector):
    """Converts a single scipy sparse vector to the required dict format."""
    coo = sparse_vector.tocoo()
    return {"values": coo.data.tolist(), "dimensions": coo.col.tolist()}

# --- Generate Hybrid Embeddings Efficiently with Batching ---
print("Generating embeddings in batches...")

# 1. Generate all dense embeddings in a single batch API call
all_dense_embeddings_raw = model.get_embeddings(corpus)
all_dense_embeddings = [emb.values for emb in all_dense_embeddings_raw]

# 2. Generate all sparse embeddings in a single batch operation
all_sparse_vectors = vectorizer.transform(corpus)

# 3. Combine all data into the final list using a list comprehension
items = [
    {
        "id": str(i),
        "title": df.title[i],
        "embedding": all_dense_embeddings[i],
        "sparse_embedding": convert_sparse_to_dict(all_sparse_vectors[i]),
    }
    for i in range(len(df))
]

print("Embedding generation complete.")

# Display the first item to verify
print("\nSample item:")
print(items[0])


In [ ]:
items = []
for i in range(len(df)):
    id = i
    title = df.title[i]
    dense_embedding = get_dense_embedding(title)
    sparse_embedding = get_sparse_embedding(title)
    items.append(
        {
            "id": id,
            "title": title,
            "embedding": dense_embedding,
            "sparse_embedding": sparse_embedding,
        }
    )
items[0]

The `get_dense_embedding()` function uses Vertex AI Embedding API for generating text embedding with 768 dimensions. This generates both dense and sparse embeddings in the following format:

```
{
  'id': 0,
  'title': 'Google Sticker',
  'embedding':
    [0.022880317643284798,
    -0.03315234184265137,
    ...
    -0.03309667482972145,
    0.04621824622154236],
  'sparse_embedding': {
    'values': [0.933008728540452, 0.359853737603667],
    'dimensions': [191, 78]
  }
}
```

The rest of the process is the same as the previous example. Upload the JSONL file to the Cloud Storage bucket, create a Vector Search index with the file, and deploy the index to the index endpoint.

In [ ]:

Import json

# This code assumes the 'items' list has been generated by the preceding cells,
# and that a 'BUCKET_URI' variable (e.g., "gs://your-bucket-name") is defined in your notebook.

# 1. Define the local filename for the JSONL output
output_filename_local = "items_hybrid.jsonl"

# 2. Write the 'items' list to the local JSONL file
# The JSONL format requires one complete JSON object per line.
print(f"Writing {len(items)} items to local file: {output_filename_local}...")
with open(output_filename_local, "w") as f:
    for item in items:
        f.write(json.dumps(item) + "\n")
print("Local file created successfully.")

# 3. Upload the local JSONL file to your Google Cloud Storage bucket
# We use the gsutil command-line tool, which is available in Vertex AI Workbench.
print(f"Uploading {output_filename_local} to {BUCKET_URI}/...")
!gsutil cp {output_filename_local} {BUCKET_URI}/
print("Upload complete.")


In [ ]:
import google.cloud.aiplatform as aiplatform

# You might also need to initialize the SDK with your project and location
# aiplatform.init(project='your-project-id', location='your-region')

# create Index (this should finish less than one minute)
my_hybrid_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-hybridsearch-index-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=10,
)


Go to **Vertex AI -> Vector Search -> Indexes (Tab)** and observe index creation.  This takes about 4 to 5 minutes.

In [ ]:
# deploy index
DEPLOYED_HYBRID_INDEX_ID = f"vs_hybridsearch_deployed_{UID}"
my_index_endpoint.deploy_index(
    index=my_hybrid_index, deployed_index_id=DEPLOYED_HYBRID_INDEX_ID
)

Go to **Vertex AI -> Vector Search -> Index Endpoints**.  Notice the Endpoint you created earlier is listed, and the deployed indexes field is working.  Click on the link for the endpoint. You see the status of the Deployed index is Deploying.  Wait, the deployment will take 30 minutes.

### While you are waiting... ###
Take time to review this [demo](https://cloud.google.com/vertex-ai/docs/vector-search/try-it) and the Additional Concepts at the end of this notebook. The demo provides a realistic example that will help you learn how Vector Search works, explore semantic and hybrid search, and see reranking in action. Submit a brief description of an animal, plant, ecommerce merchandise, or other item, and let Vector Search do the rest!

## Task 7. Run a hybrid query

After deploying the hybrid index, you can run a hybrid query:

In [ ]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    HybridQuery,
)
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from vertexai.preview.language_models import TextEmbeddingModel

# The rest of your code
# ...


For the query text `Kids`, generate both dense and sparse embeddings for the word, and encapsulate them to the `HybridQuery` object. The difference from the previous `HybridQuery` is two additional parameters: `dense_embedding` and `rrf_ranking_alpha`.

This time, we'll print distances for each item:

In [ ]:
# run a hybrid query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_HYBRID_INDEX_ID,
    queries=[query],
    num_neighbors=10,
)


import json

# This code assumes the 'items' list has been generated by the preceding cells,
# and that a 'BUCKET_URI' variable (e.g., "gs://your-bucket-name") is defined in your notebook.

# 1. Define the local filename for the JSONL output
output_filename_local = "items_hybrid.jsonl"

# 2. Write the 'items' list to the local JSONL file
# The JSONL format requires one complete JSON object per line.
print(f"Writing {len(items)} items to local file: {output_filename_local}...")
with open(output_filename_local, "w") as f:
    for item in items:
        f.write(json.dumps(item) + "\n")
print("Local file created successfully.")

# 3. Upload the local JSONL file to your Google Cloud Storage bucket
# We use the gsutil command-line tool, which is available in Vertex AI Workbench.
print(f"Uploading {output_filename_local} to {BUCKET_URI}...")
!gsutil cp {output_filename_local} {BUCKET_URI}/
print("Upload complete.")

# print results
for idx, neighbor in enumerate(response[0]):
    title = df.title[int(neighbor.id)]
    dense_dist = neighbor.distance if neighbor.distance else 0.0
    sparse_dist = neighbor.sparse_distance if neighbor.sparse_distance else 0.0
    print(f"{title:<40}: dense_dist: {dense_dist:.3f}, sparse_dist: {sparse_dist:.3f}")
    
    
    
    

In each `neighbor` object, there's a `distance` property that has the distance between the query and the item with the dense embedding, and a `sparse_distance` property that has the distance with the sparse embedding. These values are inverted distances, so a higher value means a shorter distance.

By running a query with `HybridQuery`, you get the following result:

```
Google Blue Kids Sunglasses             : dense_dist: 0.677, sparse_dist: 0.606
Google Red Kids Sunglasses              : dense_dist: 0.665, sparse_dist: 0.572
YouTube Kids Coloring Pencils           : dense_dist: 0.655, sparse_dist: 0.478
YouTube Kids Character Sticker Sheet    : dense_dist: 0.644, sparse_dist: 0.468
Google White Classic Youth Tee          : dense_dist: 0.645, sparse_dist: 0.000
Google Doogler Youth Tee                : dense_dist: 0.639, sparse_dist: 0.000
Google Indigo Youth Tee                 : dense_dist: 0.637, sparse_dist: 0.000
Google Black Classic Youth Tee          : dense_dist: 0.632, sparse_dist: 0.000
Chrome Dino Glow-in-the-Dark Youth Tee  : dense_dist: 0.632, sparse_dist: 0.000
Google Bike Youth Tee                   : dense_dist: 0.629, sparse_dist: 0.000
```

In addition to the token-based search results that have the `Kids` keyword, there are also semantic search results included. For example, `Google White Classic Youth Tee` is included because the embedding model knows that `Youth` and `Kids` are semantically similar.

To merge the token-based and semantic search results, hybrid search uses [Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/%7Egvcormac/cormacksigir09-rrf.pdf). For more information about RRF and how to specify the rrf_ranking_alpha parameter, see the later section "What is Reciprocal Rank Fusion?".

### Reranking
RRF provides a way to merge the ranking from semantic and token-based search results. In many production information retrieval or recommender systems, the results will be going through further precision ranking algorithms - so called reranking. With the combination of the millisecond level fast retrieval with vector search, and precision reranking on the results, you can build [multi-stage systems](https://cloud.google.com/blog/products/ai-machine-learning/scaling-deep-retrieval-tensorflow-two-towers-architecture?e=48754805) that provide higher search quality or recommendation performance.

![multi-stage systems](https://cloud.google.com/vertex-ai/docs/vector-search/images/reranking.png)

# Conclusion

In this tutorial, we have learned the concept of hybrid search, how to build vector search index for token-based search with sparse embeddings, and how to combine it with the semantic search with dense embeddings. As we see on the example, hybrid search provides a way to mix the best part of the two approaches, resulting in higher search quality in production systems.


# Task 8. Cleaning up (Optional)

In case you are using your own Cloud project, not a temporary project on Qwiklab, please make sure to delete all the Indexes, Index Endpoints and Cloud Storage buckets after finishing this tutorial. Otherwise the remaining objects would incur unexpected costs.


In [ ]:
# wait for a confirmation
input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# delete Indexes
my_hybrid_index.delete()

# delete Cloud Storage bucket
! gsutil rm -r "{BUCKET_URI}"

# Start using hybrid search
The following resources can help you get started with using hybrid search in Vector Search.

## Hybrid search resources
- [Input data format and structure](https://cloud.google.com/vertex-ai/docs/vector-search/setup/format-structure): Input data format for building sparse embedding index
- [Query public index to get nearest neighbors](https://cloud.google.com/vertex-ai/docs/vector-search/query-index-public-endpoint): How to run queries with hybrid search
- [Reciprocal Rank Fusion outperforms Condorcet and individual Rank Learning Methods](https://plg.uwaterloo.ca/%7Egvcormac/cormacksigir09-rrf.pdf): Discussion of the RRF algorithm

## Vector Search resources
- [Overview of Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/vector-search/overview)
- [Vector Search quickstart](https://cloud.google.com/vertex-ai/docs/vector-search/quickstart)


# Additional concepts
The following sections describe TF-IDF and TfidVectorizer, Reciprical Rank Fusion, and the alpha parameter in further detail.

## How do TF-IDF and `TfidfVectorizer` work?
The `fit_transform()` function executes two important processes of the TF-IDF algorithm:

- **Fit**: The vectorizer calculates the Inverse Document Frequency (IDF) for each term in the vocabulary. IDF reflects how important a term is across the entire corpus. Rare terms get higher IDF scores:

```
IDF(t) = log_e(Total number of documents / Number of documents containing term t)
```

- **Transform**:
 - **Tokenization**: Breaks the documents down into individual terms (words or phrases)
 - **Term Frequency (TF) Calculation**: Counts how often each term appears in each document with:

```
TF(t, d)** = (Number of times term t appears in document d) / (Total number of terms in document d)
```

 - **TF-IDF Calculation**: Combines the TF for each term with the pre-calculated IDF to create a TF-IDF score. This score represents the importance of a term in a particular document relative to the entire corpus.

```
TF-IDF(t, d) = TF(t, d) * IDF(t)
```

The TF-IDF vectorizer tries to put higher weight to signature words in the dataset, such as "Shirts" or "Dino", compared to trivial words, such as "The", "a" or "of", and counts how many times those signature words are used in the specified document. Each value of a sparse embedding represents a frequency of each word based on the counts.

## What is Reciprocal Rank Fusion?
For merging the token-based and semantic search results, hybrid search uses [Reciprocal Rank Fusion (RRF)](https://plg.uwaterloo.ca/%7Egvcormac/cormacksigir09-rrf.pdf). RRF is an algorithm for combining multiple ranked lists of items into a single, unified ranking. It's a popular technique for merging search results from different sources or retrieval methods, especially in hybrid search systems and large language models.

In case of the hybrid search of Vector Search, the dense distance and sparse distance are measured in different spaces and can't be directly compared to each other. Thus, RRF works effectively for merging and ranking the results from the two different spaces.

Here's how RRF Works:

- **Reciprocal rank**: For each item in a ranked list, calculate its reciprocal rank. This means taking the inverse of the item's position (rank) in the list. For example, the item ranked number one gets a reciprocal rank of 1/1 = 1, and the item ranked number two gets 1/2 = 0.5.
- **Sum reciprocal ranks**: Sum the reciprocal ranks for each item across all the ranked lists. This gives a final score for each item.
- **Sort by final score**: Sort the items by their final score in descending order. The items with the highest scores are considered the most relevant or important.

In short, the items with higher ranks in both dense and sparse results will be pulled up to the top of the list. Thus, the item "Google Blue Kids Sunglasses" is at the top as it has higher ranks in both dense and sparse search results. Items like "Google White Classic Youth Tee" are low ranked as they only have ranks in the dense search result.

## How the alpha parameter behaves
The example of how to use hybrid search sets the parameter `rrf_ranking_alpha` as `0.5` when creating the `HybridQuery` object. You can specify a weight on ranking the dense and sparse search results using the following values for `rrf_ranking_alpha`:

- `1`, or not specified: Hybrid search uses only dense search results and ignores sparse search results
- `0`: Hybrid search uses only sparse search results and ignores dense search results
- `0` to `1`: Hybrid search merges both results from dense and sparse with the weight specified by the value. `0.5` means they will be merged with the same weight
